In [ ]:
import torch
import gc
import logging

# del session.logger
# del session.epoch_logger
# del session
# gc.collect()
# torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=256,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=4,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.1,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=4,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=15,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(450 * 3),  # 5% total steps
    adalora_tfinal=(450 * 8),  # 50-80% total steps
    adalora_deltaT=(450 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(450 * 50),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=50,
    steps_per_epoch=450,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.6,
        "mse_loss": 0.4,
        "commitment_loss": 0.05,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
        "mmd_loss": 0.0,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 3e-4
config.batch_size = 256
config.steps_per_epoch = 450

config.brain_encoder_config.quantizer = "vq"
config.brain_encoder_config.num_codebooks = 4
config.brain_encoder_config.codebook_size = 8
config.brain_encoder_config.quantizer_commitment = 0.1
config.brain_encoder_config.transformer_input = "concat"

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/architecture/subject/vq_concat_input_4C_8",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 45, Unseen Subject: 12, Unseen Both: 4.

VQQuantizer initialized with 4 codebooks of size 8
RNNEncoder initialized as conformer with 4 layers, 256 d_model, 4 nhead
	Embedding: sinusoidal, params: 6075392
SimpleConv initialized with 9061360 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 4, hidden_dim: 256, params 2626048
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


2025-02-18 23:33:42,994	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 135/135 [08:29<00:00,  3.77s/it]


Testing done in 0.68m.
Epoch 1 done in 9.20m. 0.07m/recording.
New highest test accuracy: 0.0270, lowest final-layer loss: 2.2056, epoch 1.


Training Epoch 2: 100%|██████████| 135/135 [08:20<00:00,  3.71s/it]


Testing done in 0.67m.
Epoch 2 done in 9.02m. 0.07m/recording.


...

Testing done in 0.73m.
Epoch 43 done in 9.10m. 0.07m/recording.


Training Epoch 44: 100%|██████████| 135/135 [08:25<00:00,  3.74s/it]


Testing done in 0.68m.
Epoch 44 done in 9.10m. 0.07m/recording.


Training Epoch 45: 100%|██████████| 135/135 [08:19<00:00,  3.70s/it]


Testing done in 0.67m.
Epoch 45 done in 8.99m. 0.07m/recording.
Early stopping at epoch 45. Highest top10 at epoch 34.


Training completed. Highest epoch at 34.


unseen_subject: Acc: 0.2936, Top5: 0.5506, Top10: 0.6656
Loss: 7.6008, Mel: 5.5943, Clip: 9.1965, MSE: 0.1686, CosSim: 0.0000
FinLayer Clip: 8.5311, MSE: 0.5490, CosSim: 0.2017, MMD: 0.0000, Total: 2.0065
FinLayer Acc: 0.2357, FinLayer Top5: 0.4723, FinLayer Top10: 0.5927


unseen_task: Acc: 0.2200, Top5: 0.4560, Top10: 0.5535
Loss: 7.6707, Mel: 5.6452, Clip: 9.2779, MSE: 0.1715, CosSim: 0.0000
FinLayer Clip: 8.6084, MSE: 0.5547, CosSim: 0.2048, MMD: 0.0000, Total: 2.0255
FinLayer Acc: 0.1575, FinLayer Top5: 0.3600, FinLayer Top10: 0.4825


unseen_both: Acc: 0.2487, Top5: 0.4700, Top10: 0.5800
Loss: 7.6382, Mel: 5.6205, Clip: 9.2408, MSE: 0.1679, CosSim: 0.0000
FinLayer Clip: 8.5781, MSE: 0.5516, CosSim: 0.2035, MMD: 0.0000, Total: 2.0177
FinLayer Acc: 0.1875, FinLayer Top5: 0.3950, FinLayer Top10: 0.4900


In [ ]:
import torch
import gc
import logging

del session.logger
del session.epoch_logger
del session
gc.collect()
torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=256,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=4,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.1,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=4,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=15,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(450 * 3),  # 5% total steps
    adalora_tfinal=(450 * 8),  # 50-80% total steps
    adalora_deltaT=(450 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(450 * 50),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=50,
    steps_per_epoch=450,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.6,
        "mse_loss": 0.4,
        "commitment_loss": 0.05,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
        "mmd_loss": 0.0,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 3e-4
config.batch_size = 256
config.steps_per_epoch = 450

config.brain_encoder_config.quantizer = "vq"
config.brain_encoder_config.num_codebooks = 4
config.brain_encoder_config.codebook_size = 16
config.brain_encoder_config.quantizer_commitment = 0.1
config.brain_encoder_config.transformer_input = "concat"

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/architecture/subject/vq_concat_input_4C_16",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 45, Unseen Subject: 12, Unseen Both: 4.

VQQuantizer initialized with 4 codebooks of size 16
RNNEncoder initialized as conformer with 4 layers, 256 d_model, 4 nhead
	Embedding: sinusoidal, params: 6075392
SimpleConv initialized with 9063408 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 4, hidden_dim: 256, params 2626048
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


Training Epoch 1: 100%|██████████| 135/135 [08:25<00:00,  3.74s/it]


Testing done in 0.68m.
Epoch 1 done in 9.10m. 0.07m/recording.
New highest test accuracy: 0.0268, lowest final-layer loss: 2.1920, epoch 1.


Training Epoch 2: 100%|██████████| 135/135 [08:19<00:00,  3.70s/it]


Testing done in 0.67m.
Epoch 2 done in 9.00m. 0.07m/recording.


...

Training Epoch 29: 100%|██████████| 135/135 [08:25<00:00,  3.74s/it]


Testing done in 0.70m.
Epoch 29 done in 9.12m. 0.07m/recording.


Training Epoch 30: 100%|██████████| 135/135 [08:26<00:00,  3.75s/it]


Testing done in 0.69m.
Epoch 30 done in 9.12m. 0.07m/recording.


Training Epoch 31:  36%|███▌      | 48/135 [03:31<05:32,  3.82s/it] 

In [ ]:
# for name, param in session.model.encoder.named_parameters():
#     if "lora_A" in name or "lora_B" in name or "lora_E" in name:
#         print(name)

In [ ]:
# import time

# dataloader = session.get_dataloader(buffer_size=30, num_workers=24, max_cache_size=400)
# dataloader.start_fetching(session.dataset["train"], cache=True)

# # Process batches as they become available
# try:
#     batches, recs, start_time = 0, 0, time.time()
#     print(f"Total recordings: {len(session.dataset['train'])}")

#     while True:
#         batch = dataloader.get_recording()

#         if batch is None:
#             break

#         brain = batch.brain_segments["all"]
#         batches += brain.shape[0]
#         recs += 1

#         if recs % 10 == 0:
#             print(f'Processed {recs} recordings of {len(session.dataset["train"])}')

#     print(
#         f"Batch {batches} ({recs} recordings) processed in {time.time() - start_time:.2f}s"
#     )
#     print(
#         f"Average processing time per recording: {(time.time() - start_time) / recs:.2f}s"
#     )
#     print(
#         f"Average processing time per batch: {(time.time() - start_time) / batches:.2f}s"
#     )

# except KeyboardInterrupt:
#     print("Interrupted")
#     dataloader.stop()
# except Exception as e:
#     print("Error", e)
#     dataloader.stop()